# **Digital Signal Processing - Midterm Project**
## **Submitted by:**
| **Figure4 Members** |
|   :----:    |
| *Banta, Ian Noel M.*    |
| *Pabelico, John Paul B.*  |
| *Santander, Mark Adrian P.*   |
| *Villasor, Vince Kazer M.*  |

# **1. Libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# pd.set_option("display.max_rows", None)

# **2. Data**

## 2.1. Extraction

In [2]:
dateDF = pd.read_excel("Figure4_MidtermProject_Data.xlsx", "Sheet1", skiprows=0, nrows= 1350, usecols="A")
humiDF = pd.read_excel("Figure4_MidtermProject_Data.xlsx", "Sheet1", skiprows=0, nrows= 1350, usecols="B")
tempDF = pd.read_excel("Figure4_MidtermProject_Data.xlsx", "Sheet1", skiprows=0, nrows= 1350, usecols="C")

## 2.2. Transformation

### 2.2.1. Data Imputation

In [3]:
humiDF['Humidity (%)'] = humiDF['Humidity (%)'].interpolate()
tempDF['Temperature (°C)'] = tempDF['Temperature (°C)'].interpolate()

## 2.3. Load

In [4]:
datetemphumiDF = pd.DataFrame()
datetemphumiDF = pd.concat([dateDF, humiDF, tempDF], axis=1)

# **3. Functions**

## 3.1. Functions For Section 4.2.1 Time of day

### Filtering Data

In [ ]:
indexesOfLateNight = []
indexesOfMorning = []
indexesOfAfternoon = []
indexesOfEvening = []
 
for i in range(len(datetemphumiDF)):
    dateTimeSplit = datetemphumiDF['Date'][i].split('_')

    timeSplit = dateTimeSplit[1].split(':')
    hour = timeSplit[0]
    
    # Late Night
    if hour in ['00', '01', '02', '03', '04', '05']:
        indexesOfLateNight.append(i)
        
    # Morning
    if hour in ['06', '07', '08', '09', '10', '11']:
        indexesOfMorning.append(i)
    
    # # Afternoon
    if hour in ['12', '13', '14', '15', '16', '17']:
        indexesOfAfternoon.append(i)
    
    # # Evening
    if hour in ['18', '19', '20', '21', '22', '23']:
        indexesOfEvening.append(i)

In [5]:
def getFilteredData(get, indexesOfTargets, datetemphumiDF):
    indexesOfTargetsPerDay = []
    tempOfTargetsPerDay = []
    humiOfTargetsPerDay = []
    datesOfTargets = []
    hoursOfTargets = []

    tempList = []
    temperatures = []
    humidity = []
    datesList = []
    prevDate = None

    lastIndexFromIndexesOfTargets= indexesOfTargets[-1]
    lastElement = datetemphumiDF['Date'][lastIndexFromIndexesOfTargets]
    
    hoursList = []

    for indx in indexesOfTargets:
        dateTimeSplit = datetemphumiDF['Date'][indx].split('_')
        dateSplit = dateTimeSplit[0].split('/')
        date = dateSplit[1]
        datetimeFromDF = dateDF['Date'][indx]
        timeSplit = dateTimeSplit[1].split(':')
        time = dateTimeSplit[1]
        
        if (date != prevDate and prevDate != None) or datetimeFromDF == lastElement:
            indexesOfTargetsPerDay.append(tempList)
            tempOfTargetsPerDay.append(temperatures)
            humiOfTargetsPerDay.append(humidity)
            datesOfTargets.append(datesList)
            hoursOfTargets.append(hoursList)
            
            tempList = []
            temperatures = []
            humidity = []
            datesList = []
            hoursList = []
        
        tempList.append(indx)
        temperatures.append(datetemphumiDF['Temperature (°C)'][indx])
        humidity.append(datetemphumiDF['Humidity (%)'][indx])
        datesList.append(datetimeFromDF)
        hoursList.append(time)
        
        prevDate = date
    
    if get == 'temp':
        return zip(tempOfTargetsPerDay, datesOfTargets, hoursOfTargets)
    
    return zip(humiOfTargetsPerDay, datesOfTargets, hoursOfTargets)

### Plotting

In [6]:
def plotFilteredData(show, title, filteredData):
    plt.figure(figsize=(25,5), dpi=500, layout='tight')

    plt.title(title)
    plt.ylabel(show)
    plt.xlabel("DATE AND TIME")

    for item, date, minutes in filteredData:
        dateLabel = date[0].split('/')[1]
        plt.scatter(date, item, label=f'Oct{dateLabel}')

    plt.setp(plt.xticks(fontsize=7))
    plt.gcf().autofmt_xdate()
    plt.legend()
    plt.grid()
    plt.show()

def plotFilteredDataPerDay(show, title, filteredData):
    position = 1
    
    plt.figure(figsize=(50, 50), dpi=500)

    for item, date, minutes in filteredData:
        plt.subplot(8, 3, position)
        plt.scatter(minutes, item, c=item)
        plt.title(date[0].split('_')[0])
        position += 1

        plt.subplots_adjust(wspace=0.1, hspace=0.3)
        plt.xticks(rotation=45)
        plt.ylabel(show)
        plt.xlabel("Hour")
        plt.grid()
        
    plt.figtext(0.47, 0.90, title, fontsize=50)
    plt.show()

### Central Tendencies

In [8]:
def getMean(x):
    return np.mean(x)

def getMedian(x):
    return np.median(x)

def getMode(x):
    vals, counts = np.unique(x, return_counts=True)
    mode = np.argwhere(counts == np.max(counts))
    return vals[mode].flatten()[0]

def getVariance(x):
    return np.var(x)

def getStandardDeviation(x):
    # standard deviation represents noise and other interference.
    #  the standard deviation is not important in itself, but only in comparison to the mean.
    return np.std(x)

def getSignalToNoiseRatio(mean, std):
    return mean / std

def getCoefficientOfVariation(mean, std):
    #  a signal (or other group of measure values) with a CV of 2%, has an SNR of 50.  Better data means a higher value for the SNR and a lower value for the CV.
    return (std / mean) * 100

def getTargetData(get, indexesOfTargets, dataFrame):
    filteredData = getFilteredData(get, indexesOfTargets, dataFrame)

    targetData = []

    for values, date, minutes in filteredData:
        for item in values:
            targetData.append(item)
    
    return targetData

def getStats(get, indexesOfTargets, dataFrame):
    if get == 'temp':
        temp = getTargetData('temp', indexesOfTargets, dataFrame)

        mean = getMean(temp)
        median = getMedian(temp)
        mode = getMode(temp)
        variance = getVariance(temp)
        standardDeviation = getStandardDeviation(temp)
        snr = getSignalToNoiseRatio(mean, standardDeviation)
        cv = getCoefficientOfVariation(mean, standardDeviation)

        print(f'Mean = {mean}°C')
        print(f'Median = {median}°C')
        print(f'Mode = {mode}°C')
        print(f'Variance = {variance}')
        print(f'Standard Deviation = {standardDeviation}')
        print(f'Signal to Noise Ratio = {snr}')
        print(f'Coefficient of Variation = {cv}%')
    
    else:
        humi = getTargetData('humi', indexesOfTargets, dataFrame)

        mean = getMean(humi)
        median = getMedian(humi)
        mode = getMode(humi)
        variance = getVariance(humi)
        standardDeviation = getStandardDeviation(humi)
        snr = getSignalToNoiseRatio(mean, standardDeviation)
        cv = getCoefficientOfVariation(mean, standardDeviation)

        print(f'Mean = {mean}%')
        print(f'Median = {median}%')
        print(f'Mode = {mode}%')
        print(f'Variance = {variance}')
        print(f'Standard Deviation = {standardDeviation}')
        print(f'Signal to Noise Ratio = {snr}')
        print(f'Coefficient of Variation = {cv}%')

# **4. Analysis Goals:**

## 4.1. What is the periodicity of the data?

FFT haha

In [ ]:
# ahaha

## 4.2. What are the central tendencies of the data according to time of day, day of week, and in general?

### 4.2.1. Time of day

LATE NIGHT

In [ ]:
getStats('temp', indexesOfLateNight, datetemphumiDF)

In [ ]:
plotFilteredData('Temperature in °C', 'LATE NIGHT', getFilteredData('temp', indexesOfLateNight, datetemphumiDF))
plotFilteredDataPerDay('Temperature in °C', 'LATE NIGHT', getFilteredData('temp', indexesOfLateNight, datetemphumiDF))

In [ ]:
getStats('humi', indexesOfLateNight, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity in %', 'LATE NIGHT', getFilteredData('humi', indexesOfLateNight, datetemphumiDF))
plotFilteredDataPerDay('Humidity in %', 'LATE NIGHT', getFilteredData('humi', indexesOfLateNight, datetemphumiDF))

MORNING

In [ ]:
getStats('temp', indexesOfMorning, datetemphumiDF)

In [ ]:
plotFilteredData('Temperature in °C', 'MORNING', getFilteredData('temp', indexesOfMorning, datetemphumiDF))
plotFilteredDataPerDay('Temperature in °C', 'MORNING', getFilteredData('temp', indexesOfMorning, datetemphumiDF))

In [ ]:
getStats('humi', indexesOfMorning, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity in %', 'MORNING', getFilteredData('humi', indexesOfMorning, datetemphumiDF))
plotFilteredDataPerDay('Humidity in %', 'MORNING', getFilteredData('humi', indexesOfMorning, datetemphumiDF))

AFTERNOON

In [ ]:
getStats('temp', indexesOfAfternoon, datetemphumiDF)

In [ ]:
plotFilteredData('Temperature in °C', 'AFTERNOON', getFilteredData('temp', indexesOfAfternoon, datetemphumiDF))
plotFilteredDataPerDay('Temperature in °C', 'AFTERNOON', getFilteredData('temp', indexesOfAfternoon, datetemphumiDF))

In [ ]:
getStats('humi', indexesOfAfternoon, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity in %', 'AFTERNOON', getFilteredData('humi', indexesOfAfternoon, datetemphumiDF))
plotFilteredDataPerDay('Humidity in %', 'AFTERNOON', getFilteredData('humi', indexesOfAfternoon, datetemphumiDF))

EVENING

In [ ]:
getStats('temp', indexesOfEvening, datetemphumiDF)

In [ ]:
plotFilteredData('Temperature in °C', 'EVENING', getFilteredData('temp', indexesOfEvening, datetemphumiDF))
plotFilteredDataPerDay('Temperature in °C', 'EVENING', getFilteredData('temp', indexesOfEvening, datetemphumiDF))

In [ ]:
getStats('humi', indexesOfEvening, datetemphumiDF)

In [ ]:
plotFilteredData('Humidity in %', 'EVENING', getFilteredData('humi', indexesOfEvening, datetemphumiDF))
plotFilteredDataPerDay('Humidity in %', 'EVENING', getFilteredData('humi', indexesOfEvening, datetemphumiDF))

### 4.2.2. Day of week

Mondays To Thursdays

In [ ]:
# haha

Fridays To Sundays

In [ ]:
# haha

### 4.2.3 In general

Ewan ko

In [ ]:
# haha

## 4.3. What patterns arise from the data you are analyzing?

### 4.3.1. idk haha

In [ ]:
# hahaha

# 5. Conclusion?

## 5.1. Haha

### 5.1.1. hahaha